## Importing libraries 
also ensuring the notebook can find the dbf900 series of .py for functions

In [1]:
import pandas as pd
import codecs

##Import section to ensure the main directory is in the path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Importing from the main directory now
from dbf900_main_bytes import parse_record, yield_blocks
from dbf900_layouts_bytes import dbf900_layout
from dbf900_formats_bytes import ebc_decode

## Connecting to local file and defining blocks

In [2]:
file_path = r'C:\PublicData\Texas\TXRRC\index\dbf900.ebc' ##Local storage location
##file origin: ftp://ftpe.rrc.texas.gov/shfwba/dbf900.ebc.gz

file = open(file_path, 'rb') ##Opens the .ebc file and reads it as bytes

split_records = []  ## array for records
n  = 247 ##Number of bytes for each record in dbf900.ebc file

## Testing the location values (section 13)
Using the decoding and layout for bytes to see if the pic_signed function is working properly<br>
Also stopping the run after 50 locations

In [3]:
well_ct = 0 ##counter for number of wells
location_ct = 0 ##counter for wells with lat-long location sections

for block in yield_blocks(file, n): ##for each 
    startval = ebc_decode(block[0:2]) ##
    if startval == '01': ##check for wbroot record
        well_ct+=1 ##increase well count by one
        API = API = '42'+ ebc_decode(block[2:10]) ##api value in records '01'
    
    if startval == '13': ##checking only the pic_signed section
        location_ct+=1  ##increase location count by one
        
        layout = dbf900_layout(startval)['layout'] ##Retrieves layout for the block

        ##reading the records needs to be in bytes and not decode the whole block before parsing
        parsed_vals = parse_record(block, layout)  ##Decodes the block based on layout

        split_records.append([API,parsed_vals])  ##adds record to the array

    if location_ct > 50: ##limiting number of wells used in example
        break

## Vewing results

In [4]:
split_records

[['4200100001',
  {'RRC-TAPE-RECORD-ID': '13',
   'WB-LOC-COUNTY': 1,
   'WB-ABSTRACT': '',
   'WB-SURVEY': 'ELLIS',
   'WB-BLOCK-NUMBER': '',
   'WB-SECTION': '',
   'WB-ALT-SECTION': '',
   'WB-ALT-ABSTRACT': '',
   'WB-FEET-FROM-SUR-SECT-1': 0,
   'WB-DIREC-FROM-SUR-SECT-1': '',
   'WB-FEET-FROM-SUR-SECT-2': 0,
   'WB-DIREC-FROM-SUR-SECT-2': '',
   'WB-WGS84-LATITUDE': 32.0018943,
   'WB-WGS84-LONGITUDE': -96.0323202,
   'WB-PLANE-ZONE': 0,
   'WB-PLANE-COORDINATE-EAST': 0.0,
   'WB-PLANE-COORDINATE-NORTH': 0.0,
   'WB-VERIFICATION-FLAG': '0'}],
 ['4200100008',
  {'RRC-TAPE-RECORD-ID': '13',
   'WB-LOC-COUNTY': 1,
   'WB-ABSTRACT': '',
   'WB-SURVEY': '',
   'WB-BLOCK-NUMBER': '',
   'WB-SECTION': '',
   'WB-ALT-SECTION': '',
   'WB-ALT-ABSTRACT': '',
   'WB-FEET-FROM-SUR-SECT-1': 0,
   'WB-DIREC-FROM-SUR-SECT-1': '',
   'WB-FEET-FROM-SUR-SECT-2': 0,
   'WB-DIREC-FROM-SUR-SECT-2': '',
   'WB-WGS84-LATITUDE': 31.9798802,
   'WB-WGS84-LONGITUDE': -96.0097968,
   'WB-PLANE-ZONE': None,